In [9]:
import pandas as pd
import numpy as np
import csv
import re
import nltk
import string

from ast import literal_eval
from tqdm import tqdm
from nltk.corpus import stopwords

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stwan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

## Remove CLS SEP

In [103]:
file_path = 'runs/02/attn_scores/stormfront_neg_attn_scores_concise.csv'

In [104]:
df = pd.read_csv(file_path, converters={"key": literal_eval})
df

,key,avg_attn_score,num_occur
0,"([CLS], [CLS])",0.826333,7601
1,"([CLS], also)",0.530325,131
2,"([CLS], ,)",0.387578,3335
3,"([CLS], it)",0.563805,1528
4,"([CLS], ')",0.717430,2808
...,...,...,...
1535791,"(sue, for)",0.644453,1
1535792,"(sue, racism)",0.797215,1
1535793,"(for, sue)",1.784629,1
1535794,"(racism, should)",0.740428,1


In [105]:
df.sort_values(by=['avg_attn_score'], inplace=True, ascending=False)
df = df.reset_index(drop=True)
df

,key,avg_attn_score,num_occur
0,"([SEP], [SEP])",8.749429,7601
1,"(starvation, [SEP])",7.564120,1
2,"(themes, [SEP])",7.381782,1
3,"(tubes, [SEP])",7.283553,1
4,"(badger, [SEP])",7.097707,1
...,...,...,...
1535791,"(##dine, ill)",0.000119,1
1535792,"(william, ill)",0.000113,1
1535793,"(williamsburg, ill)",0.000107,1
1535794,"(fur, ill)",0.000071,1


In [106]:
drop_idx = []
for i, (a, b) in tqdm(enumerate(df['key'])):
    if a == b:
        drop_idx.append(i)
        continue
    if a == '[SEP]' or b == '[SEP]' or a == '[CLS]' or b == '[CLS]':
        drop_idx.append(i)
df = df.drop(drop_idx)

1535796it [00:01, 1485297.42it/s]


In [107]:
filterd_df = df[df['num_occur'] > 10]
filterd_df

,key,avg_attn_score,num_occur
4886,"(ll, ')",2.250511,167
4951,"(., agreed)",2.240600,14
6420,"(., disgusting)",2.036507,16
6507,"(##8, 250)",2.027680,32
6616,"(hope, ')",2.015377,36
...,...,...,...
1535029,"(##ith, _)",0.001642,30
1535037,"(##and, _)",0.001635,30
1535052,"(##led, up)",0.001610,12
1535089,"(##bic, _)",0.001562,30


In [108]:
token1_list = []
token2_list = []
for token1, token2 in filterd_df['key']:
    token1_list.append(token1)
    token2_list.append(token2)
filterd_df['token_1'] = token1_list
filterd_df['token_2'] = token2_list
filterd_df = filterd_df.drop(columns='key')
filterd_df

C:\Users\stwan\AppData\Local\Temp/ipykernel_19640/2490702253.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filterd_df['token_1'] = token1_list
C:\Users\stwan\AppData\Local\Temp/ipykernel_19640/2490702253.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filterd_df['token_2'] = token2_list


,avg_attn_score,num_occur,token_1,token_2
4886,2.250511,167,ll,'
4951,2.240600,14,.,agreed
6420,2.036507,16,.,disgusting
6507,2.027680,32,##8,250
6616,2.015377,36,hope,'
...,...,...,...,...
1535029,0.001642,30,##ith,_
1535037,0.001635,30,##and,_
1535052,0.001610,12,##led,up
1535089,0.001562,30,##bic,_


In [109]:
drop_token_list = ["'", "/", "’", "-", "_", ",", ".", ")", "(", "!", ":", "?"]

for token in drop_token_list:
    filterd_df = filterd_df[filterd_df['token_2'] != token]
    filterd_df = filterd_df[filterd_df['token_1'] != token]

filterd_df.head(20)

,avg_attn_score,num_occur,token_1,token_2
6507,2.027680,32,##8,250
11125,1.621500,13,tr,##udeau
13428,1.495758,32,250,##8
18689,1.308614,16,*,bull
19394,1.289940,19,power,white
24450,1.177135,50,##l,lo
27704,1.122840,11,whites,black
29259,1.100318,12,kids,black
29745,1.093603,12,the,reich
29818,1.092666,11,##s,traitor


In [110]:
filterd_df.to_csv(file_path[:-4] + '_filtered.csv', index=False)

## Remove Punc Stop Words

In [27]:
file_path = 'runs/02/attn_scores/filtered/stormfront_neg_attn_scores_concise_filtered.csv'

In [28]:
df = pd.read_csv(file_path)
df

,avg_attn_score,num_occur,token_1,token_2
0,2.027680,32,##8,250
1,1.621500,13,tr,##udeau
2,1.495758,32,250,##8
3,1.308614,16,*,bull
4,1.289940,19,power,white
...,...,...,...,...
40825,0.003999,12,2001,london
40826,0.003991,17,[,##za
40827,0.003990,12,##ez,com
40828,0.002088,12,lab,up


In [29]:
punc_list = list(string.punctuation)
stop_word_list = stopwords.words('english')

In [30]:
drop_idx = []
for i, (token_1, token_2) in tqdm(enumerate(zip(df['token_1'], df['token_2']))):
    if token_1 in punc_list or token_2 in punc_list or token_1 in stop_word_list or token_2 in stop_word_list:
        drop_idx.append(i)
df = df.drop(drop_idx)

40830it [00:00, 176753.11it/s]


In [31]:
df.to_csv(file_path[:-4] + '_filtered.csv', index=False)